ExE Interaction Analysis with NHAMES Data
-----------------------------------------


<h4>Analysis 1: Main Effect Filter</h4>
<ol>
<li>Create the Discovery and Replicate Datasets</li>
<li>Define Covariantes and Outcome</li>
<li>Run ExE Interaction to Discovery Dataset</li>
<li>Run Correct_PValue Function</li>
<li>Filter LRT_PVALUE_FDR < 0.1</li>
<li>Get Terms Pairs result after Filter</li>
-------------------------------------------
<li>Loop each Term Pair filted</li>
<li>Run ExE Interaction to Replicate Datase with each pair</li>
.....
</ol>

<h4>Analysis 2: Knowlegde Drive Filter</h4>
<ol>
<li>Create the Discovery and Replicate Datasets</li>
</ol>


<h3>Part 1: Main Effect Filter</h3>

In [10]:
# Packages
import os
import django
import pandas as pd
import igem
from pathlib import Path

In [11]:
# Set up Django environment
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "igem.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

# Variables
v_path_current = Path.cwd()

In [12]:
# Load NHAMES File with exposomes
# Here we are load Discovery and Replicate dataset filted by year
path_nhames_disc = str(v_path_current) + "/part1_discovery_FDR.txt"
path_nhames_repl = str(v_path_current) + "/part1_replication_FDR.txt"
df_nhames_disc = igem.epc.load.from_tsv(path_nhames_disc)
df_nhames_repl = igem.epc.load.from_tsv(path_nhames_repl)

Loaded 9,063 observations of 63 variables
Loaded 9,874 observations of 63 variables


In [13]:
# Setup columns rules
# list of Outcomes
list_outcome = ["logBMI"]

# list of Covariants
list_covariant = [
    "female",
    "black",
    "mexican",
    "other_hispanic",
    "other_eth",
    "SDDSRVYR",
    "SES_LEVEL",
    "RIDAGEYR",
]

In [57]:
# Run Interation Study to Discovery Data
df_interactions_disc = igem.epc.analyze.interaction_study(
    data=df_nhames_disc,
    outcomes=list_outcome,
    # covariates=list_covariant,
)

InteractionRegression
-------------------------
Continuous Outcome (family = Gaussian): 'logBMI'
Using 9,063 of 9,063 observations
Regressing 62 variables
	0 binary variables
	0 categorical variables
	62 continuous variables
	0 genotypes variables
Processing 1,891 interactions
-------------------------
Running 1,891 interactions using 12 processes...


	Finished Running 1,891 interactions
42 tests had an error
	LBXVCF:URXUTL = NULL due to: too few complete observations (min_n filter: 102 < 200)
	LBXVCF:SMD057 = NULL due to: too few complete observations (min_n filter: 155 < 200)
	LBXVCF:URXP24 = NULL due to: too few complete observations (min_n filter: 139 < 200)
	LBXVCF:URXUPT = NULL due to: too few complete observations (min_n filter: 102 < 200)
	LBXVTO:URXUTL = NULL due to: too few complete observations (min_n filter: 102 < 200)
	LBXVTO:URXP24 = NULL due to: too few complete observations (min_n filter: 134 < 200)
	LBXVTO:URXUPT = NULL due to: too few complete observations (min_n filter: 102 < 200)
	URXUTL:LBXD05 = NULL due to: too few complete observations (min_n filter: 0 < 200)
	URXUTL:LBXF04 = NULL due to: too few complete observations (min_n filter: 0 < 200)
	URXUTL:LBXHXC = NULL due to: too few complete observations (min_n filter: 0 < 200)
	URXUTL:LBX170 = NULL due to: too few complete observations (min_n filter: 0 < 200)
	UR

In [58]:
df_interactions_disc = df_interactions_disc[df_interactions_disc['LRT_pvalue'] < 0.1]
df_corrested_pvalues_disc = igem.epc.analyze.add_corrected_pvalues(
    data=df_interactions_disc,
    pvalue="LRT_pvalue",
)
# TODO: This return None!

In [19]:
# Filter to keep only LRT_PVALUE_FDR < 0.1
df_fdr_disc = df_interactions_disc[df_interactions_disc['LRT_pvalue'] < 0.1]

In [22]:
# Get only Term1 and Term2 to run Interaction in Replicate Dataset
df_fdr_disc.reset_index(inplace=True)
ls_exposome = igem.epc.modify.colfilter(df_fdr_disc, only=('Term1', 'Term2'))

Running colfilter
--------------------------------------------------------------------------------
Keeping 2 of 10 variables:
	0 of 0 binary variables
	0 of 0 categorical variables
	0 of 7 continuous variables
	2 of 3 unknown variables


Start ExE Interactions on Replicate Dataset

==> Use ExE Pairwise from Discovery Interactios saved in ls_exposomes

In [49]:
ls_inter = []
for e in ls_exposome.itertuples():
    e1 = e[1]
    e2 = e[2]
    # Run Interation Study to Replicate Data
    df_inter = igem.epc.analyze.interaction_study(
        data=df_nhames_repl,
        outcomes=list_outcome,
        interactions=[(e1, e2)],
        # covariates=list_covariant,
    )
    df_inter.reset_index(inplace=True)
    # Save as List to High Performance
    ls_inter.append(
        [
            df_inter.iloc[0][0],
            df_inter.iloc[0][1],
            df_inter.iloc[0][2],
            df_inter.iloc[0][3],
            df_inter.iloc[0][4],
            df_inter.iloc[0][5],
            df_inter.iloc[0][6],
            df_inter.iloc[0][7],
            df_inter.iloc[0][8],
        ]
    )


InteractionRegression
-------------------------
Continuous Outcome (family = Gaussian): 'logBMI'
Using 9,874 of 9,874 observations
Regressing 62 variables
	0 binary variables
	0 categorical variables
	62 continuous variables
	0 genotypes variables
Processing 1 interactions
-------------------------
Running 1 interactions using 12 processes...


	Finished Running 1 interactions
0 tests had an error
Completed Interaction Study for logBMI

Completed association study
InteractionRegression
-------------------------
Continuous Outcome (family = Gaussian): 'logBMI'
Using 9,874 of 9,874 observations
Regressing 62 variables
	0 binary variables
	0 categorical variables
	62 continuous variables
	0 genotypes variables
Processing 1 interactions
-------------------------
Running 1 interactions using 12 processes...
	Finished Running 1 interactions
0 tests had an error
Completed Interaction Study for logBMI

Completed association study
InteractionRegression
-------------------------
Continuous Outcome (family = Gaussian): 'logBMI'
Using 9,874 of 9,874 observations
Regressing 62 variables
	0 binary variables
	0 categorical variables
	62 continuous variables
	0 genotypes variables
Processing 1 interactions
-------------------------
Running 1 interactions using 12 processes...
	Finished Running 1 interactions
0 tests had an error
Completed In

In [51]:
# Return to DataFrame
df = pd.DataFrame(ls_inter, columns=['Term1', 'Term2', 'Outcome', 'Conv', 'N', 'Beta', 'SE', 'Beta_pvalue', 'LRT_pvalue'])
print(df)

                 Term1     Term2 Outcome  Conv     N  Beta  SE  Beta_pvalue  \
0                LBXHA    LBXHPE  logBMI  True  1405   NaN NaN          NaN   
1                LBXHA    LBXCRY  logBMI  True  9205   NaN NaN          NaN   
2               LBXRBF  DR1TMFAT  logBMI  True  8970   NaN NaN          NaN   
3    cigarette_smoking    LBXHPE  logBMI  True   866   NaN NaN          NaN   
4               LBXRBF  DR1TM181  logBMI  True  8970   NaN NaN          NaN   
..                 ...       ...     ...   ...   ...   ...  ..          ...   
452             LBXRPL  DR1TTFAT  logBMI  True  8689   NaN NaN          NaN   
453             LBXCOT  DR1TM181  logBMI  True  8945   NaN NaN          NaN   
454             LBXVTO    LBXHXC  logBMI  True   760   NaN NaN          NaN   
455             LBXD05    LBXALC  logBMI  True  1329   NaN NaN          NaN   
456           DR1TMFAT    LBX194  logBMI  True  1249   NaN NaN          NaN   

       LRT_pvalue  
0    3.219465e-03  
1    4.7073